# K-Nearest Neighbors Classification from Scratch with NumPy

K-nearest neighhbors (KNN) is a machine learning algorithm that can be used both for classification and regression purposes. It falls under the category of supervised learning algorithms that predicts target values for unseen observations. In other words, it operates on labelled datasets and predicts either a class (classification) or a numeric value (regression) for the test data.

In this post, I will be implementing KNN classification algorithm from scratch using NumPy library only. As I have mentioned in my previous posts, I won't be using an already implemented version of knn algorithm with the help of sklearn, I will be implementing it with only NumPy and will be using other libraries for creating and using datasets and data visualization purposes.

Considering there are many useful resources out there that explains the fundamentals of knn, I want to get into coding part immediately. Let's do some classification!



In [2]:
import numpy as np
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt

We start by importing the libraries and functions that will be used throughout the implementation:
* `numpy`: It is used for numerical computation of multidimensional arrays, obviously.
* `make_classification`: We will use this to create our very own classification dataset. We can decide how many classes and features we want, whether we want samples to be clustered and etc.
* `matplotlib`: It will be used when we want to visualize the dataset.

In [3]:
def euclidian_distance(a, b):
    return np.sqrt(np.sum((a-b)**2, axis=1))

In knn algorithm, we will need a function to calculate the distances between training data points and the data that we would like to classify. Here, I've chosen the euclidian distance as it is the widely used type in machine learning applications. One can try classifying with other distance metrics such as Manhattan distance, Chebychev  distance and etc. 

In [4]:
def kneighbors(X_test, return_distance=False):
       
        dist = []
        neigh_ind = []
        
        point_dist = [euclidian_distance(x_test, X_train) for x_test in X_test]

        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[:n_neighbors]
    
            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]
    
            dist.append(dist_list)
            neigh_ind.append(ind_list)
        
        if return_distance:
            return np.array(dist), np.array(neigh_ind)
        
        return np.array(neigh_ind)

In practical aspect of knn, we try to find the neighbors, the closest data points to the data that we want to classify. Whether a data point is close or not is determined by our euclidian distance function implemented above. Here, the actual value of the distance between data points does not matter, rather, we are interested in the order of those distances. We pick a number as our hyperparameter (k) before the training process and pick k-neareset neighbors to our data points we want to classify during training. Hence, for instance when we say 5-nearest neighbors, we mean the first 5 closest data points. 

In the `kneighbors` function above, we find the distances between each point in test dataset (the data points we want to classify) and the rest of the dataset, that is the training data. We store those distances in `point_dist` in which each row corresponds to a list of distances between one test data point and all of the training data. Hence, we go over each row, enumerate it and then sort it according to the distances. The reason we enumerate each row is because we don't want to lose the indices of training data points that we calculated the distances with, since we are going to refer them later. 

Consequently, `sorted_neigh` holds the first k-nearest neighbors of our test data points and they are sorted according to their euclidian distances. We, then, extract indices and distance values from `sorted_neigh` and return them.

In [5]:
def predict(X_test, weights='uniform'):
        
        class_num = 3
        
        if weights=='uniform':
            neighbors = kneighbors(X_test)
            y_pred = np.array([np.argmax(np.bincount(y_train[neighbor])) for neighbor in neighbors])
        
            return y_pred 
    
        if weights=='distance':
        
            dist, neigh_ind = kneighbors(X_test, return_distance=True)
        
            inv_dist = 1/dist
            
            mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis]
            
            proba = []
            
            for i, row in enumerate(mean_inv_dist):
                
                row_pred = self.y_train[neigh_ind[i]]
                
                for k in range(class_num):
                    indices = np.where(row_pred==k)
                    prob_ind = np.sum(row[indices])
                    proba.append(np.array(prob_ind))
        
            predict_proba = np.array(proba).reshape(X_test.shape[0], class_num)
            
            y_pred = np.array([np.argmax(item) for item in predict_proba])
            
            return y_pred

After finding the k-nearest neighbors, we try to predict the classes that our test data points belong to. Here, we have k neighbors and each neighbor has a vote in deciding the class label. However, voting mechanism may vary according to the chosen criterion. Here, in the `predict` function above, if `weights` are chosen as `uniform` it means that each neighbor has the equal vote (weight) in deciding the class label irrespective of their distances. 

Let's say we have 5-nearest neighbors of our test data point, 3 of them belong to class A and 2 of them belong to class B. We disregard the distances of neighbors and conclude that the test data point belongs to the class A since the majority of neighbors are a part of class A. However, if `weights` are chosen as `distance`, then this means the distances of neighbors do matter, indeed. Hence, whichever neighbor that is closest to the test data point has more weight (vote) proportional to the inverse of their distances. So regarding the aforementioned example, if those 2 points belonging the class A are a lot closer to the test data point than the other 3 points, then, that may play a bigger role in deciding the class label for data point.

In `predict` function, it is quite easy to predict the label for data points if `weights` are `uniform`. First, we get the indices of neighbors and then use those indices to get their corresponding class labels from training dataset. Each row in `neighbors` corresponds to the set of neighbors that each test data point has. We then find the occurences of class labels using `numpy`'s `bincount` function and get the index of the maximum occurence which corresponds to the predicted class label. 

Things get a little messier if we have `weights` chosen as `distance`. In this case, we find the mean inverse of neighbor distances and calculate class probabilities for each test data point. 

In [6]:
def score(X_test, y_test):
        y_pred = predict(X_test)
        
        return float(sum(y_pred == y_test))/ float(len(y_test))

I have implemented the `score` function as a very simple accuracy metric used in classification problems widely. We simply return the percentage of correctly classified labels. It is pretty straightforward! 

In [7]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples = 1000, n_features=2, n_redundant=0, n_informative=2,
                             n_clusters_per_class=1, n_classes=3, random_state=21)


mu = np.mean(X, 0)
sigma = np.std(X, 0)

X = (X - mu ) / sigma

Now, it is time to create the dataset that we will be testing our knn algorithm upon. We make use of `sklearn.dataset`'s `make_classification` function to populate the dataset. Afterwards, we normalize the each data point by subtracking the mean and then dividing by the standard deviation.

In [8]:
n_neighbors = 5

data = np.hstack((X, y[:, np.newaxis]))
        
np.random.shuffle(data)

split_rate = 0.7

train, test = np.split(data, [int(split_rate*(data.shape[0]))])

X_train = train[:,:-1]
y_train = train[:, -1]

X_test = test[:,:-1]
y_test = test[:, -1]

y_train = y_train.astype(int)
y_test = y_test.astype(int)

After creating the dataset, we simply implement a random split for our training phase, hence obtain our training and test sets.

In [9]:
kneighbors(X_test)

array([[698, 488, 341, 122, 306],
       [317, 539, 621, 321, 206],
       [  8, 322,  66,  63, 626],
       ...,
       [680, 584, 270, 236, 686],
       [ 57, 357, 277, 448, 628],
       [  2, 382, 566, 269,  55]])

It is time to test our code of knn implementation. We get the k-nearest neighbors of our test data set. Do notice that, each row is related to each data point in our test set and elements in each row correspond to the indices of neighbors of the test data point.

In [10]:
predict(X_test)

array([1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 2, 0, 1, 1, 0, 2, 1, 2, 1, 0, 0, 1, 0, 0, 1, 2, 0,
       2, 2, 0, 0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 1, 0, 0, 2, 0, 2, 2, 0, 2,
       1, 1, 2, 0, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1,
       2, 1, 2, 1, 0, 1, 1, 1, 2, 1, 2, 0, 0, 2, 2, 2, 0, 2, 0, 1, 1, 0,
       2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 1, 0, 2, 1, 1, 2, 1,
       2, 2, 0, 1, 1, 2, 2, 0, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 0, 2, 0, 1,
       1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 2, 0, 1, 2, 2, 1, 1, 1, 0, 1, 2, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 2, 1, 1, 2, 1, 2, 0, 2, 2, 2, 2, 1, 0, 1,
       2, 2, 1, 1, 1, 0, 0, 0, 2, 1, 2, 0, 0, 1, 1, 1, 2, 1, 0, 1, 1, 0,
       2, 0, 0, 0, 1, 1, 1, 0, 2, 2, 0, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0,
       1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 2, 1, 2, 2, 0, 0, 1, 0,
       0, 1, 2, 1, 1, 0, 1, 2, 2, 0, 0, 2, 1, 2, 1, 1, 0, 2, 1, 1, 2, 1,
       1, 1, 2, 2, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2])

As expected, `predict` function outputs the predicted class labels of the test data.

In [11]:
score(X_test, y_test)

0.9733333333333334

Turns out, our implementation did a really good job given the high accuracy score. 

## Class Implementation of KNN

In [12]:
import numpy as np

class KNearestNeighbors():
    def __init__(self, X_train, y_train, n_neighbors=5, weights = 'uniform'):
    
        self.X_train = X_train
        self.y_train = y_train
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        
        self.class_num = 3

        
    def euclidian_distance(self, a, b):     
        return np.sqrt(np.sum((a-b)**2, axis=1))
    
    
    
    def kneighbors(self, X_test, return_distance=False):
       
        dist = []
        neigh_ind = []
        
        point_dist = [self.euclidian_distance(x_test, self.X_train) for x_test in X_test]

        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[:self.n_neighbors]
    
            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]
    
            dist.append(dist_list)
            neigh_ind.append(ind_list)
        
        if return_distance:
            return np.array(dist), np.array(neigh_ind)
        
        return np.array(neigh_ind)
         
    
    def predict(self, X_test):
        
        if self.weights=='uniform':
            neighbors = self.kneighbors(X_test)
            y_pred = np.array([np.argmax(np.bincount(self.y_train[neighbor])) for neighbor in neighbors])
        
            return y_pred 
    
        if self.weights=='distance':
        
            dist, neigh_ind = self.kneighbors(X_test, return_distance=True)
        
            inv_dist = 1/dist
            
            mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis]
            
            proba = []
            
            for i, row in enumerate(mean_inv_dist):
                
                row_pred = self.y_train[neigh_ind[i]]
                
                for k in range(self.class_num):
                    indices = np.where(row_pred==k)
                    prob_ind = np.sum(row[indices])
                    proba.append(np.array(prob_ind))
        
            predict_proba = np.array(proba).reshape(X_test.shape[0], self.class_num)
            
            y_pred = np.array([np.argmax(item) for item in predict_proba])
            
            return y_pred
            
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        
        return float(sum(y_pred == y_test))/ float(len(y_test))

After implementing all necessary functions, it is pretty easy to create the class implementation of knn. Here only newly added function is the `__init__` function for our class. 

## Comparing Our Implementation with Sklearn’s KNeighborsClassifier